In [251]:
import numpy as np
import scipy as cp
import pandas as pd

import matplotlib.pyplot as plt

import sys
import docplex.mp
from docplex.mp.model import Model # import the Model class from docplex.mp

In [252]:
url = None
key = None

In [253]:
m = Model(name='SLPRD')

In [254]:
# Capita Adequacy Constraint:0.08 *(0.09*B+*0.09*R) <=0.5

In [255]:
# Leverage Ratio constraint: 0.03*(L+B+R)-E<=0.3

In [256]:
# Net Stabe Funding Constraint: 0.5*L+0.65*0.04*B+0.65*0.03*R-0.95*0.03*D-0.5*0.03*C<= 0.4

In [257]:
# Liquidity Coverage ratio: -0.04*L-0.51*0.03*D <=0

In [258]:
# Define Asset side variables
L = m.continuous_var (name="L")
B = m.continuous_var (name="B")
R = m.continuous_var (name="R")
# Define Liability side variables
C = m.continuous_var (name="C")
D = m.continuous_var (name="D")
T = m.continuous_var (name="T")
E = m.continuous_var (name="E")
# Adjustment Cost
a = m.continuous_var (name="a")

In [259]:
a=0.02

In [260]:
#Capita Adequacy Constraint
m.add_constraint(0.08*0.09*B+0.08*0.09*R<=0.5)

docplex.mp.LinearConstraint[](0.007B+0.007R,LE,0.5)

In [261]:
# Leverage Ratio constraint:
m.add_constraint(0.03*(L+B+R)-E<=0.3)

docplex.mp.LinearConstraint[](0.030L+0.030B+0.030R-E,LE,0.3)

In [262]:
# Net Stabe Funding Constraint:
m.add_constraint(0.5*L+0.65*0.04*B+0.65*0.03*R-0.95*0.03*D-0.5*0.03*C<= 0.4)

docplex.mp.LinearConstraint[](0.500L+0.026B+0.019R-0.015C-0.028D,LE,0.4)

In [263]:
# Liquidity Coverage ratio:
m.add_constraint(0.04*L-0.51*0.03*D <=0)

docplex.mp.LinearConstraint[](0.040L-0.015D,LE,0)

In [264]:
#Objective function deterministic :
#Max ((0.04 L - a |L-0.1|)+ (0.033 B -a |B-0.5|)+ (0.024R-a|R-0.4|)+(0.0025C-a|C-0.1|)+(0.0051D-a|D-0.4*|)
#-(0.03*E-a|E-0.3|+ 0.04*T-a|T-0.2|] )

In [265]:
# Define Absolute Value  variables
U1= m.continuous_var (name="U1")
U2= m.continuous_var (name="U2")
U3= m.continuous_var (name="U3")
V1= m.continuous_var (name="V1")
V2= m.continuous_var (name="V2")
V3= m.continuous_var (name="V3")
V4= m.continuous_var (name="V4")

In [266]:
#Absolute Value constraint:-(L-0.1) <= U1 <= (L-0.1) 

In [267]:
#Absolute Value constraint:-(B-0.5) <= U2 <= (B-0.5) 

In [268]:
#Absolute Value constraint:-(R-0.4) <= U3 <= (R-0.4) 

In [269]:
#Absolute Value constraint:-(C-0.1) <= V1 <= (C-0.1) 

In [270]:
#Absolute Value constraint:-(D-0.4) <= V2 <= (D-0.4) 
#Absolute Value constraint:-(E-0.3) <= V3 <= (E-0.3) 
#Absolute Value constraint:-(T-0.2) <= V4 <= (T-0.2) 

In [271]:
#Absolute Value constraints after the rearrangement
m.add_constraint(U1-L<= -0.1)
m.add_constraint(-L-U1<= -0.1)
m.add_constraint(-B-U2<= -0.5)
m.add_constraint(U2-B<= -0.5)
m.add_constraint(-R-U3<= -0.4)
m.add_constraint(U3-R<= -0.4)
m.add_constraint(-C-V1<= -0.1)
m.add_constraint(V1-C<= -0.1)
m.add_constraint(-D-V2<= -0.4)
m.add_constraint(V2-D<= -0.4)
m.add_constraint(V3-E<= -0.3)
m.add_constraint(V3-E<= -0.3)
m.add_constraint(-E-V3<= -0.3)
m.add_constraint(-T-V4<= -0.2)
m.add_constraint(V4-T<= -0.2)

docplex.mp.LinearConstraint[](-T+V4,LE,-0.2)

In [272]:
#Nonnegativity
m.add_constraint(L >= 0)
m.add_constraint(B >= 0)
m.add_constraint(R >=0)
m.add_constraint(C<= 0.2)
m.add_constraint(D <= 0.6)
m.add_constraint(E >= 0)
m.add_constraint(T >= 0)

docplex.mp.LinearConstraint[](T,GE,0)

In [273]:
m.maximize ((0.04* L - a*U1)+ (0.033* B -a *U2)+ (0.024*R-a* U3)+(0.0025*C-a*V1)+(0.0051*D-a*V2)-(0.03*E-a*V3)-(0.04*T-a*V4))
m.print_information()

Model: SLPRD
 - number of variables: 15
   - binary=0, integer=0, continuous=15
 - number of constraints: 26
   - linear=26
 - parameters: defaults
 - objective: maximize
 - problem type is: LP


In [274]:
msol = m.solve(url=url, log_output=True)
msol.report()

Version identifier: 20.1.0.0 | 2020-11-10 | 9bedb6d68
CPXPARAM_Read_DataCheck                          1
Tried aggregator 1 time.
LP Presolve eliminated 16 rows and 6 columns.
Aggregator did 2 substitutions.
Reduced LP has 8 rows, 7 columns, and 19 nonzeros.
Presolve time = 0.02 sec. (0.02 ticks)

Iteration log . . .
Iteration:     1   Scaled dual infeas =             0.053360
Iteration:     3   Dual objective     =             0.519464


AttributeError: 'SolveSolution' object has no attribute 'report'